Imports

In [ ]:
import ga_polynomial as ga
import plotly.express as px
from math import ceil
import numpy as np
import json
import plotly.graph_objects as go
import logging
from tqdm import tqdm
import matplotlib as plt

logging.getLogger().setLevel(logging.WARN)

Setup

In [ ]:
count = 100
individualLength = 6
individualMin = -50
individualMax = 50

retain = 0.1
mutate = 0.01
random_select = 0.1

### Running with new roulette wheel

In [ ]:
maxGen = 100
fitted = 1

population = ga.createPopulation(count, individualMin, individualMax, individualLength)
fitness_history = [ga.getAvgPopulationFitness(population, errorMethod='mse')]

while fitted == 0 and len(fitness_history) < maxGen:

    population = ga.evolve(population, 
                        retain=retain, 
                        mutate=mutate, 
                        select='roulette',
                        errorMethod='mse')

    fitted = ga.getAvgPopulationFitness(population, errorMethod='mse')
    fitness_history.append(lastGrade)

#results[', '.join([str(retain), str(mutate), str(random_select)])] = filledArray

In [ ]:
individual = ga.createIndividual(-50, 50, 6)
population = ga.createPopulation(5, individualMin, individualMax, individualLength)

In [ ]:
individual

In [ ]:
population

In [ ]:
np.append(population, [individual], axis=0)

In [ ]:
errors = np.array([])

# Get total fitness of population (cumulative sum of each individual's error)
for individual in population:

    # Get errors for each individual
    errors = np.append(errors, ga.getMSE(individual))

# Get fitness, append to list of fitnesses
fitnesses, _ = ga.errorsToFitness(errors)

# Calculate selection probability for each individual as a proportion of total fitness for population
selectProbabilities = np.divide(fitnesses, np.sum(fitnesses))

In [ ]:
selectProbabilities

In [ ]:
from random import random, uniform
poolSize = 20

matingPool = np.array([])

# Select individuals to add to mating pool using probability based on fitness of individual
for individual, selectProbability in zip(population, selectProbabilities):
    if selectProbability > random():
        matingPool = np.append(matingPool, individual)
    else:
        pass

    # Break when mating pool has reached size specified by retain parameter
    if len(matingPool) >= poolSize:
        logging.info("Breaking from crossover loop with mating pool size {}.".format(len(matingPool)))
        break
    else:
        continue

In [ ]:
from random import random, uniform

In [ ]:
uniform(0,1/0.8)

In [ ]:
matingPool.astype(int)

### Trying to Plot Polynomials

In [ ]:
maxGen = 500
polynomials = {}
checkpoints = np.linspace(100, maxGen, int(maxGen/100))

p = population(count, individualMin, individualMax, individualLength)
fitness_history = [grade(p)]
lastGrade = 1
it = 0
x = np.linspace(-1,1, 20)

while lastGrade != 0 and len(fitness_history) < maxGen:

    p, individualGrades = evolve(p, 
               retain=retain, 
               random_select=random_select, 
               mutate=mutate)

    lastGrade = grade(p)
    fitness_history.append(lastGrade)
    
    if it in checkpoints:

        # Rank population based on individual grades
        bestIndividual = sorted(individualGrades)[0][1]
        
        # Extract polynomial coefficients from individual
        a1 = bestIndividual[0]
        a2 = bestIndividual[1]
        a3 = bestIndividual[2]
        a4 = bestIndividual[3]
        a5 = bestIndividual[4]
        c = bestIndividual[5]
        
        y = []
        
        # For a range of values of x, calculate the target and predicted value of y
        for element in x:
            yPredict = a1*element**5 + a2*element**4 + a3*element**3 + a4*element**2 + a5*element + c
            
            # Store target and prediction as pair in list of tuples
            y.append(yPredict)
        
        polynomials[it] = (x, np.array(y))
        
        figPolynomial = px.line(polynomials)
        figPolynomial.show()
        
    it += 1

#results[', '.join([str(retain), str(mutate), str(random_select)])] = filledArray

In [ ]:
np.array(y)

---

In [ ]:
fig = px.line(fitness_history)
fig.show()

In [ ]:
population

In [ ]:
data = []

for key in results.keys():
    droppedNones = list(filter(None, results[key]))
    lowestVal = min(droppedNones)
    data.append((lowestVal, key))
    
bestPerformers = sorted(data)[:10]
bestPerformers

In [ ]:
maxIterations = 1000
resultsBest = {}

for _, params in bestPerformers:
    
    # Get params from list of best performers
    paramList = params.split(', ')
    retain = float(paramList[0])
    mutate = float(paramList[1])
    random_select = float(paramList[2])
    
    logging.warning("retain: {}; mutate: {}; random_select: {}\n".format(retain, mutate, random_select))
    
    # Create and grade first population
    p = population(count, individualMin, individualMax, individualLength)
    fitness_history = [grade(p)]
    lastGrade = 1

    # Until you do max iterations or reach error = 0
    while lastGrade != 0 and len(fitness_history) < maxIterations:

        # Create population
        p = evolve(p, 
                   retain=retain, 
                   random_select=random_select, 
                   mutate=mutate)

        # Grade population
        lastGrade = grade(p)
        fitness_history.append(lastGrade)

        # Fill with Nones to make all same length
        filledArray = [None]*maxIterations
        lastIndex = len(fitness_history)
        filledArray[:lastIndex-1] = fitness_history

    # Store population results in new dictionary for best performers (length 10)
    resultsBest[', '.join([str(retain), str(mutate), str(random_select)])] = filledArray

In [ ]:
with open("results_polynomial_broadSweep.json", "w") as outfile:  
    json.dump(results, outfile) 
with open("results_polynomial_bestPerformers.json", "w") as outfile:  
    json.dump(resultsBest, outfile) 

---

In [ ]:
import json
with open('results_polynomial_bestPerformers.json') as json_file: 
    resultsBest = json.load(json_file)

In [ ]:
import plotly.express as px
fig = px.line(resultsBest)
fig.show()